In [1]:
import duckdb
import pandas as pd

orders = pd.read_csv("orders.csv")
returns = pd.read_csv("returns.csv")

df = orders.merge(returns[["order_id","returned"]], on="order_id", how="left")
df["is_returned"] = df["returned"].apply(lambda x: 1 if x=="Yes" else 0)

# Run your SQL queries directly
category_stats = duckdb.query("""
    SELECT category,
           COUNT(*) AS total_orders,
           SUM(is_returned) AS total_returns,
           ROUND(100.0*SUM(is_returned)/COUNT(*),2) AS return_rate
    FROM df
    GROUP BY category
    ORDER BY return_rate DESC
""").to_df()

supplier_stats = duckdb.query("""
    SELECT supplier,
           COUNT(*) AS total_orders,
           SUM(is_returned) AS total_returns,
           ROUND(100.0*SUM(is_returned)/COUNT(*),2) AS return_rate
    FROM df
    GROUP BY supplier
""").to_df()

print(category_stats)
print(supplier_stats)



      category  total_orders  total_returns  return_rate
0     Clothing            10           10.0        100.0
1       Beauty            10           10.0        100.0
2       Sports            10           10.0        100.0
3  Electronics            10            0.0          0.0
4    Furniture            10            0.0          0.0
     supplier  total_orders  total_returns  return_rate
0  Supplier_B            10            6.0         60.0
1  Supplier_C            10            6.0         60.0
2  Supplier_D            10            6.0         60.0
3  Supplier_A            10            6.0         60.0
4  Supplier_E            10            6.0         60.0


In [3]:
# ---- Return rate per region ----
region_stats = duckdb.query("""
    SELECT region,
           COUNT(*) AS total_orders,
           SUM(is_returned) AS total_returns,
           ROUND(100.0*SUM(is_returned)/COUNT(*),2) AS return_rate
    FROM df
    GROUP BY region
    ORDER BY return_rate DESC
""").to_df()

print(region_stats)


# ---- Return rate per channel ----
channel_stats = duckdb.query("""
    SELECT channel,
           COUNT(*) AS total_orders,
           SUM(is_returned) AS total_returns,
           ROUND(100.0*SUM(is_returned)/COUNT(*),2) AS return_rate
    FROM df
    GROUP BY channel
    ORDER BY return_rate DESC
""").to_df()

print(channel_stats)


  region  total_orders  total_returns  return_rate
0  South            13            8.0        61.54
1  North            13            8.0        61.54
2   West            12            7.0        58.33
3   East            12            7.0        58.33
  channel  total_orders  total_returns  return_rate
0  Retail            23           14.0        60.87
1  Online            27           16.0        59.26


In [4]:
# Save category return rates
category_stats.to_csv("category_return_rate.csv", index=False)

# Save supplier return rates
supplier_stats.to_csv("supplier_return_rate.csv", index=False)

# Save region return rates
region_stats.to_csv("region_return_rate.csv", index=False)

# Save channel return rates
channel_stats.to_csv("channel_return_rate.csv", index=False)


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Features and target
X = df[["category","supplier","region","channel","price"]]
y = df["is_returned"]

# Preprocess (encode categoricals)
categorical_cols = ["category","supplier","region","channel"]
numeric_cols = ["price"]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numeric_cols)
    ]
)

# Build logistic regression pipeline
model = Pipeline(steps=[("preprocessor", preprocessor),
                        ("classifier", LogisticRegression(max_iter=1000))])

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)

# Predict probability of return
df["return_probability"] = model.predict_proba(X)[:,1]


In [6]:
high_risk = df[df["return_probability"] > 0.5][
    ["order_id","product_id","category","supplier","region","channel","price","return_probability"]
]

high_risk.to_csv("high_risk_products.csv", index=False)
